# Bac à Sable
Expérimentation autour de différentes fonctionnalités

In [1]:
from numpy.lib.stride_tricks import as_strided
import numpy as np
import time
from scipy.signal import convolve2d

## Détection de configuration de capture:

In [ ]:
k_diags = np.array([0.25 * np.array([[-1, 0, 0, 0],
                                    [0, 1, 0, 0],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, -1]]),
                    0.25 * np.array([[0, 0, 0, -1],
                                    [0, 0, 1, 0],
                                    [0, 1, 0, 0],
                                    [-1, 0, 0, 0]])])

k_lines = [0.25 * np.array([[-1, 1, 1, -1]]), 0.25 * np.array([[-1],[1],[1],[-1]])]

#k_free_threes = [np.ones((1,6)), np.ones((6,1)), np.identity(6), np.rot90(np.identity(6))]
k_free_threes = [np.array([1, 2, 2, 2, 2, 1]),
                 np.array([[1], [2], [2], [2], [2], [1]]),
                 np.array([[1, 0, 0, 0, 0, 0],
                           [0, 2, 0, 0, 0, 0],
                           [0, 0, 2, 0, 0, 0],
                           [0, 0, 0, 2, 0, 0],
                           [0, 0, 0, 0, 2, 0],
                           [0, 0, 0, 0, 0, 1]]),
                 np.array([[0, 0, 0, 0, 0, 1],
                           [0, 0, 0, 0, 2, 0],
                           [0, 0, 0, 2, 0, 0],
                           [0, 0, 2, 0, 0, 0],
                           [0, 2, 0, 0, 0, 0],
                           [1, 0, 0, 0, 0, 0]])]

In [ ]:
arr = np.random.randint(1,5, size=(10, 10))

In [ ]:
arr

array([[1, 3, 4, 3, 3, 3, 4, 2, 1, 1],
       [1, 4, 3, 3, 3, 3, 2, 1, 2, 3],
       [4, 1, 3, 4, 3, 4, 3, 4, 3, 4],
       [4, 4, 3, 4, 2, 2, 2, 3, 3, 1],
       [1, 1, 3, 2, 3, 1, 1, 2, 4, 4],
       [4, 4, 4, 3, 3, 1, 4, 3, 4, 1],
       [3, 4, 3, 1, 4, 1, 4, 4, 1, 3],
       [1, 2, 2, 2, 4, 4, 1, 1, 4, 3],
       [3, 2, 2, 4, 2, 3, 2, 1, 2, 1],
       [4, 1, 4, 3, 3, 4, 1, 2, 4, 2]])

In [ ]:
extend_arr = np.zeros((16,16))
extend_arr[3:-3, 3:-3] = arr
extend_arr

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 3., 4., 3., 3., 3., 4., 2., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 4., 3., 3., 3., 3., 2., 1., 2., 3., 0., 0., 0.],
       [0., 0., 0., 4., 1., 3., 4., 3., 4., 3., 4., 3., 4., 0., 0., 0.],
       [0., 0., 0., 4., 4., 3., 4., 2., 2., 2., 3., 3., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 3., 2., 3., 1., 1., 2., 4., 4., 0., 0., 0.],
       [0., 0., 0., 4., 4., 4., 3., 3., 1., 4., 3., 4., 1., 0., 0., 0.],
       [0., 0., 0., 3., 4., 3., 1., 4., 1., 4., 4., 1., 3., 0., 0., 0.],
       [0., 0., 0., 1., 2., 2., 2., 4., 4., 1., 1., 4., 3., 0., 0., 0.],
       [0., 0., 0., 3., 2., 2., 4., 2., 3., 2., 1., 2., 1., 0., 0., 0.],
       [0., 0., 0., 4., 1., 4., 3., 3., 4., 1., 2., 4., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [39]:
c1 = [0,0]
c2 = [9,0]
c3 = [0,9]
c4 = [9,9]
c5 = [5,3]
c6 = [6,8]

In [40]:
view1 = extend_arr[c1[0]:c1[0]+3+1, c1[1]+3]
view2 = extend_arr[c1[0]+3:c1[0]+2*3+1, c1[1]+3]
view3 = extend_arr[c1[0]+3, c1[1]:c1[1]+3+1]
view4 = extend_arr[c1[0]+3, c1[1]+3:c1[1]+2*3+1]

In [41]:
print(view1) # colonne
print(view2) # colonne
print(view3) # ligne
print(view4) # ligne


[0. 0. 0. 1.]
[1. 1. 4. 4.]
[0. 0. 0. 1.]
[1. 3. 4. 3.]


In [42]:
conv_c  = np.multiply(view1, k_lines[0])
conv_cc = np.multiply(view2, k_lines[0])
conv_l  = np.multiply(view3, k_lines[0])
conv_ll = np.multiply(view4, k_lines[0])

In [43]:
print(conv_c)
print(conv_cc)
print(conv_l)
print(conv_ll)

[[-0.    0.    0.   -0.25]]
[[-0.25  0.25  1.   -1.  ]]
[[-0.    0.    0.   -0.25]]
[[-0.25  0.75  1.   -0.75]]


In [44]:
res_c = np.sum(conv_c)
res_cc = np.sum(conv_cc)
res_l = np.sum(conv_l)
res_ll = np.sum(conv_ll)

In [45]:
print(res_c)
print(res_cc)
print(res_l)
print(res_ll)

-0.25
0.0
-0.25
0.75


In [46]:
view5 = extend_arr[c1[0]:c1[0] + 3 + 1, c1[1]:c1[1]+3+1]
view6 = extend_arr[c1[0]:c1[0] + 3 + 1, c1[1]+3:c1[1] + 2 * 3  + 1]
view7 = extend_arr[c1[0] + 3:c1[0] + 2 * 3 + 1, c1[1]:c1[1] + 3 + 1]
view8 = extend_arr[c1[0] + 3:c1[0] + 2 * 3 + 1, c1[1] + 3:c1[1] + 2 * 3 + 1]

In [47]:
print(view5)
print(view6)
print(view7)
print(view8)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 3. 4. 3.]]
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 4.]
 [0. 0. 0. 4.]]
[[1. 3. 4. 3.]
 [1. 4. 3. 3.]
 [4. 1. 3. 4.]
 [4. 4. 3. 4.]]


In [ ]:
np.sum(np.multiply(extend_grid[yx[0]:yx[0] + 3 + 1, yx[1]:yx[1] + 3 + 1], k_capture))
np.sum(np.multiply(extend_grid[yx[0]:yx[0] + 3 + 1, yx[1] + 3:yx[1] + 2 * 3  + 1], k_capture))
np.sum(np.multiply(extend_grid[yx[0] + 3:yx[0] + 2 * 3 + 1, yx[1]:yx[1] + 3 + 1], k_capture))
np.sum(np.multiply(extend_grid[yx[0] + 3:yx[0] + 2 * 3 + 1, yx[1] + 3:yx[1] + 2 * 3 + 1], k_capture))

In [48]:
sub_shape = k_diags[0].shape
print(sub_shape)

(4, 4)


In [49]:
view_shape = tuple(np.subtract(sub_arr.shape, sub_shape) + 1) + sub_shape

NameError: name 'sub_arr' is not defined

In [27]:
np.subtract(sub_arr.shape, sub_shape) + 1

array([4, 4])

In [19]:
view_shape

(4, 4, 4, 4)

In [36]:
arr_view = as_strided(sub_arr, view_shape, sub_arr.strides * 2)

True

In [31]:
sub_arr.strides * 2

(80, 8, 80, 8)

In [24]:
arr_view

array([[[[-1.,  0.,  0.,  0.],
         [-1.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.]],

        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  1., -1.]],

        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  1., -1., -1.]],

        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 1., -1., -1.,  1.]]],


       [[[-1.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  1., -1.],
         [ 0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  1., -1., -1.],
         [ 0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 1., -1., -1.,  1.],
         [ 0.,  0.,  0.

In [50]:
def strided4D(arr,arr2,s):
    strided = np.lib.stride_tricks.as_strided
    s0,s1 = arr.strides
    m1,n1 = arr.shape
    m2,n2 = arr2.shape    
    out_shp = (1+(m1-m2)//s, m2, 1+(n1-n2)//s, n2)
    return strided(arr, shape=out_shp, strides=(s*s0,s*s1,s0,s1))

In [51]:
test_arr = np.arange(1,26)

In [52]:
test_arr.reshape(5,5)

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [53]:
k = np.identity(3)

In [54]:
strided4D(test_arr.reshape(5, 5), k, 2)

array([[[[ 1,  2,  3],
         [ 6,  7,  8]],

        [[ 3,  4,  5],
         [ 8,  9, 10]],

        [[ 5,  6,  7],
         [10, 11, 12]]],


       [[[11, 12, 13],
         [16, 17, 18]],

        [[13, 14, 15],
         [18, 19, 20]],

        [[15, 16, 17],
         [20, 21, 22]]]])

In [55]:
strided4D(test_arr, k, 2)

ValueError: not enough values to unpack (expected 2, got 1)

In [56]:
def _subboard_4_Conv2D(grid, k_shape:tuple, stride:tuple) -> np.array:
    """ Generates the sub view of the grid to be multiply with the kernel.
    First the shape of the sub_grid array is calculated, it depends on
    the grid shape and the kernel shape.
    The sub_grid array shape will be (n_x, n_y, k_x, k_y) with:
        * n_x: number of application of the kernel along row (with stride of 1)
        * n_y: number of application of the kernel along column (with stride of 1)
        * k_x, k_y: the shape of the kernel
    In this way sub_grid is a numpy array of n_x/n_y rows/columns of (k_x x k_y)
    sub view of the grid.
    Args:
    -----
        k_shape ([tuple[int]]): shape of the kernel
        stride ([tuple(int)]): put self.grid.strides * 2 (but why?)
    """
    view_shape = tuple(np.subtract(grid.shape, k_shape) + 1) + k_shape
    sub_grid = as_strided(grid, view_shape, stride * 2)
    return sub_grid

def _my_conv2D(grid, kernel:np.array) -> np.array:
    """ Retrieves the sub_grid from the function _subboard_4_Conv2D and performs
    the convolution (array multiplication + einstein sum along the 3rd and 4th
    dimensions).
    Args:
    -----
        * kernel ([np.array]): the kernel to use for convolution.
    """
    sub_grid = _subboard_4_Conv2D(grid, k_shape=kernel.shape, stride=grid.strides)
    res_conv = np.multiply(sub_grid, kernel)
    convolved = np.einsum('ijkl->ij', res_conv)
    return convolved.astype('int8')

In [57]:
def check_board(grid):
    """[summary]
    """
    ## Checking if white pair captured
    # Checking the diagonal:
    conv_diag1 = _my_conv2D(k_diags[0])
    conv_diag2 = _my_conv2D(k_diags[1])
    # Checking vertical and horizontal
    conv_lin1 = _my_conv2D(k_lines[0])
    conv_lin2 = _my_conv2D(k_lines[1])
    
    coord_cd1 = np.argwhere(conv_diag1 == 1)
    coord_cd2 = np.argwhere(conv_diag2 == 1)
    coord_cl1 = np.argwhere(conv_lin1 == 1)
    coord_cl2 = np.argwhere(conv_lin2 == 1)
    if coord_cd1.shape[0] != 0:
        for coord in coord_cd1:
            print("[check_board] - conv_diag1")
            grid[coord[0] + 1][coord[1] + 1] = 0
            grid[coord[0] + 2][coord[1] + 2] = 0
    if coord_cd2.shape[0] != 0:
        for coord in coord_cd2:
            print("[check_board] - conv_diag2")
            grid[coord[0] + 1][coord[1] + 2] = 0
            grid[coord[0] + 2][coord[1] + 1] = 0
    if coord_cl1.shape[0] != 0:
        for coord in coord_cl1:
            print("[check_board] - conv_lin1")
            grid[coord[0]][coord[1] + 1] = 0
            grid[coord[0]][coord[1] + 2] = 0
    if coord_cl2.shape[0] != 0:
        for coord in coord_cl2:
            print("[check_board] - conv_lin2")
            grid[coord[0] + 1][coord[1]] = 0
            grid[coord[0] + 2][coord[1]] = 0
    ## Checking if black pair captured
    # Checking the diagonal:
    conv_diag1 = _my_conv2D(-1 * k_diags[0])
    conv_diag2 = _my_conv2D(-1 * k_diags[1])
    # Checking vertical and horizontal
    conv_lin1 = _my_conv2D(-1 * k_lines[0])
    conv_lin2 = _my_conv2D(-1 * k_lines[1])
    print("<<<== CONV_LIN1 ==>>>")
    print(conv_lin1)
    print("<<<== CONV_LIN2 ==>>>")
    print(conv_lin2)
    print("<<<== ========= ==>>>")
    coord_cd1 = np.argwhere(conv_diag1 == 1)
    coord_cd2 = np.argwhere(conv_diag2 == 1)
    coord_cl1 = np.argwhere(conv_lin1 == 1)
    coord_cl2 = np.argwhere(conv_lin2 == 1)
    if coord_cd1.shape[0] != 0:
        for coord in coord_cd1:
            print("[check_board] - conv_diag1 -1")
            grid[coord[0] + 1][coord[1] + 1] = 0
            grid[coord[0] + 2][coord[1] + 2] = 0
    if coord_cd2.shape[0] != 0:
        for coord in coord_cd2:
            print("[check_board] - conv_diag2 -1")
            grid[coord[0] + 1][coord[1] + 2] = 0
            grid[coord[0] + 2][coord[1] + 1] = 0
    if coord_cl1.shape[0] != 0:
        for coord in coord_cl1:
            print("[check_board] - conv_lin1 -1")
            grid[coord[0]][coord[1] + 1] = 0
            grid[coord[0]][coord[1] + 2] = 0
    if coord_cl2.shape[0] != 0:
        for coord in coord_cl2:
            print("[check_board] - conv_lin2 -1")
            grid[coord[0] + 1][coord[1]] = 0
            grid[coord[0] + 2][coord[1]] = 0

In [58]:
def issimplefreethree_position(yx, grid):
    """[summary]
    Args:
    yx ([type]): [description]
    grid ([type]): [description]
    Returns:
    ___: [description]
    """
    tmp = np.zeros((18,18))
    tmp[yx[0] + 4, yx[1] + 4] = 1
    tmp[4:-4, 4:-4] += grid + 1
    r_start, r_end = yx[0], yx[0] + 8
    c_start, c_end = yx[1], yx[1] + 8
    
    print(tmp)
    # Convolution sur la ligne 
    view_l = tmp[yx[0] + 4, yx[1]:yx[1] + 9]
    #print(">=== view_l ===<\n", view_l)
    res_l = [np.sum(np.multiply(view_l[i:i+6], k_free_threes[0])) for i in range(4)]
    #print(">=== res_l ===<\n", res_l)
    
    #Convolution sur la colonne:
    view_c = tmp[yx[0]:yx[0] + 9, yx[1] + 4]
    #print(">=== view_c ===<\n", view_c)
    res_c = [np.sum(np.multiply(view_c[i:i+6], k_free_threes[1].flatten())) for i in range(4)]
    #print(">=== res_c ===<\n", res_c)
    
    # Convolution sur diagonale descendante gauche-droite
    view_d1 = [tmp[yx[0]+i:yx[0]+6+i, yx[1]+i:yx[1]+6+i] for i in range(4)]
    #print(">=== view_d1 ===<\n", np.array(view_d1))
    res_d1 = [np.sum(np.multiply(view_d1[i], k_free_threes[2])) for i in range(4)]
    #print(">=== res_d1 ===<\n", res_d1)
    
    # Convolution sur diagonale montante gauche-droite
    view_d2 = [tmp[yx[0] +3 - i:yx[0] + 9 -i, yx[1]+i:yx[1]+6+i] for i in range(4)]
    #print(">=== view_d2 ===<\n", view_d2)
    res_d2 = [np.sum(np.multiply(view_d2[i], k_free_threes[3])) for i in range(4)]
    #print(">=== res_d2 ===<\n", res_d2)
    res = [*res_l, *res_c, *res_d1, *res_d2]
    if any([np.any(arr >= 16) for arr in res]):
        count = np.count_nonzero(np.array(res) >=16)
        return True, res, count
    return False, res
            

In [59]:
grid1 = np.zeros((10,10))
grid2 = np.zeros((10,10))
grid3 = np.zeros((10,10))
grid4 = np.zeros((10,10))
grid5 = np.zeros((10,10))
grid6 = np.zeros((10,10))
grid7 = np.zeros((10,10))

In [60]:
grid1[0:2,2] = 1

grid2[3:5,2] = 1

grid3[5,0:2] = 1

grid4[6,3:5] = 1

#grid5[[3,4],[3,4]] = 1
grid5[[6, 6], [7, 8]] = 1

grid6[[5, 6], [5, 6]] = 1

grid7[[2, 5],[2, 5]] = 1


In [61]:
grid7

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [62]:
issimplefreethree_position([3,3], grid7)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

(True,
 [11.0,
  12.0,
  12.0,
  12.0,
  11.0,
  12.0,
  12.0,
  12.0,
  13.0,
  15.0,
  16.0,
  15.0,
  11.0,
  12.0,
  12.0,
  11.0],
 1)

## Détection des doubles free threes

In [26]:
# kernels to check if there is a double free threes
S = 1 # choose a number corresponding to what the stone will be multiply by
V = 1 # choose a number corresponding to what the empty position will be multiply by
s1_a = np.array([[V, 0, 0, 0, 0, 0, 0],
				 [0, S, 0, 0, 0, 0, 0],
				 [0, 0, S, 0, 0, 0, 0],
				 [0, 0, V, S, S, S, V],
				 [0, 0, 0, 0, V, 0, 0]], dtype='int8')
s1_b = np.array([[0, V, 0, 0, 0],
				 [0, S, 0, 0, 0],
				 [0, S, 0, 0, 0],
				 [V, S, S, S, V],
				 [0, V, 0, 0, 0]], dtype='int8')
s1_c = np.array([[0, 0, 0, 0, V],
				 [0, 0, 0, S, 0],
				 [0, 0, S, 0, 0],
				 [V, S, S, S, V],
				 [V, 0, 0, 0, 0]], dtype='int8')
s1_d = np.array([[V, 0, 0, 0, 0, 0, V],
				 [0, S, 0, 0, 0, S, 0],
				 [0, 0, S, 0, S, 0, 0],
				 [0, 0, 0, S, 0, 0, 0],
				 [0, 0, V, 0, V, 0, 0]], dtype='int8')

s2_a = np.array([[V, 0, 0, 0, 0, 0, 0, 0],
				 [0, S, 0, 0, 0, 0, 0, 0],
				 [0, 0, S, 0, 0, 0, 0, 0],
				 [0, 0, 0, V, 0, 0, 0, 0],
				 [0, 0, 0, V, S, S, S, V],
				 [0, 0, 0, 0, 0, V, 0, 0]], dtype='int8')
s2_b = np.array([[0, V, 0, 0, 0],
				 [0, S, 0, 0, 0],
				 [0, S, 0, 0, 0],
				 [0, V, 0, 0, 0],
				 [V, S, S, S, V],
				 [0, V, 0, 0, 0]], dtype='int8')
s2_c = np.array([[0, 0, 0, 0, 0, V],
				 [0, 0, 0, 0, S, 0],
				 [0, 0, 0, S, 0, 0],
				 [0, 0, V, 0, 0, 0],
				 [V, S, S, S, V, 0],
				 [V, 0, 0, 0, 0, 0]], dtype='int8')
s2_d = np.array([[V, 0, 0, 0, 0, 0, 0, 0],
				 [0, S, 0, 0, 0, 0, 0, V],
				 [0, 0, S, 0, 0, 0, S, 0],
				 [0, 0, 0, V, 0, S, 0, 0],
				 [0, 0, 0, 0, S, 0, 0, 0],
				 [0, 0, 0, V, 0, V, 0, 0]], dtype='int8')

s3_a = np.array([[V, 0, 0, 0, 0, 0, 0, 0],
				 [0, S, 0, 0, 0, 0, 0, 0],
				 [0, 0, V, 0, 0, 0, 0, 0],
				 [0, 0, 0, S, 0, 0, 0, 0],
				 [0, 0, 0, V, S, S, S, V],
				 [0, 0, 0, 0, 0, V, 0, 0]], dtype='int8')
s3_b = np.array([[0, V, 0, 0, 0],
				 [0, S, 0, 0, 0],
				 [0, V, 0, 0, 0],
				 [0, S, 0, 0, 0],
				 [V, S, S, S, V],
				 [0, V, 0, 0, 0]], dtype='int8')
s3_c = np.array([[0, 0, 0, 0, 0, V],
				 [0, 0, 0, 0, S, 0],
				 [0, 0, 0, V, 0, 0],
				 [0, 0, S, 0, 0, 0],
				 [V, S, S, S, V, 0],
				 [V, 0, 0, 0, 0, 0]], dtype='int8')
s3_d = np.array([[V, 0, 0, 0, 0, 0, 0, 0],
				 [0, S, 0, 0, 0, 0, 0, V],
				 [0, 0, V, 0, 0, 0, S, 0],
				 [0, 0, 0, S, 0, S, 0, 0],
				 [0, 0, 0, 0, S, 0, 0, 0],
				 [0, 0, 0, V, 0, V, 0, 0]], dtype='int8')

s4_a = np.array([[V, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, S, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, V, 0, 0, 0, 0, 0],
                 [0, 0, 0, V, S, V, S, S, V],
                 [0, 0, 0, 0, 0, V, 0, 0, 0]], dtype='int8')
s4_b = np.array([[0, V, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0],
                 [0, V, 0, 0, 0, 0],
                 [V, S, V, S, S, V],
                 [0, V, 0, 0, 0, 0]], dtype='int8')
s4_c = np.array([[0, 0, 0, 0, 0, V],
                 [0, 0, 0, 0, S, 0],
                 [0, 0, 0, S, 0, 0],
                 [0, 0, V, 0, 0, 0],
                 [V, S, V, S, S, V],
                 [V, 0, 0, 0, 0, 0]], dtype='int8')
s4_d = np.array([[V, 0, 0, 0, 0, 0, 0, 0, V],
                 [0, S, 0, 0, 0, 0, 0, S, 0],
                 [0, 0, S, 0, 0, 0, S, 0, 0],
                 [0, 0, 0, V, 0, V, 0, 0, 0],
                 [0, 0, 0, 0, S, 0, 0, 0, 0],
                 [0, 0, 0, V, 0, V, 0, 0, 0]], dtype='int8')

s5_a = np.array([[V, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, S, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, V, 0, 0, 0, 0, 0],
                 [0, 0, 0, V, S, S, V, S, V],
                 [0, 0, 0, 0, 0, V, 0, 0, 0]], dtype='int8')
s5_b = np.array([[0, V, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0],
                 [0, V, 0, 0, 0, 0],
                 [V, S, S, V, S, V],
                 [0, V, 0, 0, 0, 0]], dtype='int8')
s5_c = np.array([[0, 0, 0, 0, 0, V],
                 [0, 0, 0, 0, S, 0],
                 [0, 0, 0, S, 0, 0],
                 [0, 0, V, 0, 0, 0],
                 [V, S, S, V, S, V],
                 [V, 0, 0, 0, 0, 0]], dtype='int8')
s5_d = np.array([[V, 0, 0, 0, 0, 0, 0, 0, V],
                 [0, S, 0, 0, 0, 0, 0, S, 0],
                 [0, 0, S, 0, 0, 0, V, 0, 0],
                 [0, 0, 0, V, 0, S, 0, 0, 0],
                 [0, 0, 0, 0, S, 0, 0, 0, 0],
                 [0, 0, 0, V, 0, V, 0, 0, 0]], dtype='int8')

s6_a = np.array([[V, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, V, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, S, 0, 0, 0, 0, 0],
                 [0, 0, 0, V, S, S, V, S, V],
                 [0, 0, 0, 0, 0, V, 0, 0, 0]], dtype='int8')
s6_b = np.array([[0, V, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0],
                 [0, V, 0, 0, 0, 0],
                 [0, S, 0, 0, 0, 0],
                 [V, S, S, V, S, V],
                 [0, V, 0, 0, 0, 0]], dtype='int8')
s6_c = np.array([[0, 0, 0, 0, 0, V],
                 [0, 0, 0, 0, S, 0],
                 [0, 0, 0, V, 0, 0],
                 [0, 0, S, 0, 0, 0],
                 [V, S, S, V, S, V],
                 [V, 0, 0, 0, 0, 0]], dtype='int8')
s6_d = np.array([[V, 0, 0, 0, 0, 0, 0, 0, V],
                 [0, S, 0, 0, 0, 0, 0, S, 0],
                 [0, 0, V, 0, 0, 0, V, 0, 0],
                 [0, 0, 0, S, 0, S, 0, 0, 0],
                 [0, 0, 0, 0, S, 0, 0, 0, 0],
                 [0, 0, 0, V, 0, V, 0, 0, 0]], dtype='int8')


In [27]:
dct_S = {'S1': {'a': s1_a, 'b': s1_b, 'c': s1_c, 'd':s1_d},
         'S2': {'a': s2_a, 'b': s2_b, 'c': s2_c, 'd':s2_d},
         'S3': {'a': s3_a, 'b': s3_b, 'c': s3_c, 'd':s3_d},
         'S4': {'a': s4_a, 'b': s4_b, 'c': s4_c, 'd':s4_d},
         'S5': {'a': s5_a, 'b': s5_b, 'c': s5_c, 'd':s5_d},
         'S6': {'a': s6_a, 'b': s6_b, 'c': s6_c, 'd':s6_d}}

In [28]:
def R_transformation(arr:np.array) -> list:
    l_kernel = []
    for k in [0, 1, 2, 3]:
        l_kernel.append(np.rot90(arr, k))
    return l_kernel

def mR_transformation(arr:np.array) -> list:
    l_kernel = []
    for k in [0, 1, 2, 3]:
        l_kernel.append(np.rot90(np.flipud(arr), k))
    return l_kernel

In [29]:
table_transf = {'S1': {'a': [True, True],
					   'b': [True, False],
        			   'c': [True, True],
              		   'd': [True, False]},
                'S2': {'a': [True, True],
					   'b': [True, True],
					   'c': [True, True],
					   'd': [True, True]},
                'S3': {'a': [True, True],
					   'b': [True, True],
					   'c': [True, True],
					   'd': [True, True]},
                'S4': {'a': [True, True],
					   'b': [True, False],
					   'c': [True, True],
					   'd': [True, False]},
                'S5': {'a': [True, True],
					   'b': [True, True],
					   'c': [True, True],
					   'd': [True, True]},
                'S6': {'a': [True, True],
					   'b': [True, False],
					   'c': [True, True],
					   'd': [True, False]}}

In [30]:
full_kernels = []
for first_k in table_transf.keys():
    for second_k in table_transf[first_k].keys():
        sym1, sym2 = table_transf[first_k][second_k]
        if sym1:
            [full_kernels.append(transf) for transf in R_transformation(dct_S[first_k][second_k])]
        if sym2:
            [full_kernels.append(transf) for transf in mR_transformation(dct_S[first_k][second_k])]

In [13]:
def _subboard_4_Conv2D(grid, k_shape:tuple, stride:tuple) -> np.array:
    """ Generates the sub view of the grid to be multiply with the kernel.
    First the shape of the sub_grid array is calculated, it depends on
    the grid shape and the kernel shape.
    The sub_grid array shape will be (n_x, n_y, k_x, k_y) with:
        * n_x: number of application of the kernel along row (with stride of 1)
        * n_y: number of application of the kernel along column (with stride of 1)
        * k_x, k_y: the shape of the kernel
    In this way sub_grid is a numpy array of n_x/n_y rows/columns of (k_x x k_y)
    sub view of the grid.
    Args:
    -----
        k_shape ([tuple[int]]): shape of the kernel
        stride ([tuple(int)]): put self.grid.strides * 2 (but why?)
    """
    view_shape = tuple(np.subtract(grid.shape, k_shape) + 1) + k_shape
    sub_grid = as_strided(grid, view_shape, stride * 2)
    return sub_grid

def _my_conv2D(grid, kernel:np.array) -> np.array:
    """ Retrieves the sub_grid from the function _subboard_4_Conv2D and performs
    the convolution (array multiplication + einstein sum along the 3rd and 4th
    dimensions).
    Args:
    -----
        * kernel ([np.array]): the kernel to use for convolution.
    """
    sub_grid = _subboard_4_Conv2D(grid, k_shape=kernel.shape, stride=grid.strides)
    res_conv = np.multiply(sub_grid, kernel)
    convolved = np.einsum('ijkl->ij', res_conv)
    return convolved.astype('int8')

In [76]:
v_my_conv2D = np.vectorize(_my_conv2D, signature='(m,n),(p,q)->()')

In [ ]:
v_my_conv2D(grid, full_kernels)

In [74]:
grid = np.zeros((19,19), dtype = 'int8')
grid[[3, 4, 6, 6, 6],[3, 4, 6, 7, 8]] = 1

In [64]:
res = _my_conv2D(grid, full_kernels[0])

In [66]:
print(grid.shape)
print(full_kernels[0].shape)
print(res.shape)

(19, 19)
(5, 7)
(15, 13)


In [34]:
res = []
start = time.time()
for kernel in full_kernels:
    res.append(_my_conv2D(grid[], kernel))
end = time.time()
print(end-start)

0.01781296730041504


In [7]:
BLACK = 1
WHITE = -1

k_freethree = np.array([1, 2, 2, 2, 2, 1])

def get_line_idx(yx:np.array):
    return (np.ones((1,9)) * yx[0]).astype('int8'), (np.arange(-4, 5) + yx[1]).astype('int8')


def get_col_idx(yx:np.array):
    return (np.arange(-4, 5) + yx[0]).astype('int8'), (np.ones((1,9)) * yx[1]).astype('int8')


def get_diag1_idx(yx:np.array):
    return (np.arange(-4, 5) + yx[0]).astype('int8'), (np.arange(-4, 5) + yx[1]).astype('int8')


def get_diag2_idx(yx:np.array):
    return (np.arange(4, -5, -1) + yx[0]).astype('int8'), (np.arange(-4, 5) + yx[1]).astype('int8')

def isdoublefreethree_position(yx:np.array, grid:np.array, color:int) -> bool:
        """[summary]
        Args:
            yx ([type]): [description]
            grid ([type]): [description]
        Returns:
            bool: [description]
        """
        pad_width = 5
        c = color
        extend_grid = np.pad(grid + c, pad_width, "constant", constant_values = (0))
        extend_grid[yx[0] + pad_width, yx[1] + pad_width] = 2 * c
        print(extend_grid)
        res = []
        res.append(np.convolve(extend_grid[get_line_idx(yx + pad_width)].reshape(-1,), c * k_freethree, "valid"))
        res.append(np.convolve(extend_grid[get_col_idx(yx + pad_width)].reshape(-1,), c * k_freethree, "valid"))
        res.append(np.convolve(extend_grid[get_diag1_idx(yx + pad_width)], c * k_freethree, "valid"))
        res.append(np.convolve(extend_grid[get_diag2_idx(yx + pad_width)], c * k_freethree, "valid"))
        nb_free_three = 0
        for r_conv in res:
            if (r_conv >= 16).any():
                nb_free_three += 1
        if nb_free_three > 1:
            return True
        return False

In [8]:
grid8 = np.zeros((10,10))
grid8[[2, 3, 5, 5],[2, 3, 6, 7]] = 1
grid8

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
stone = 1
yx = np.array([5, 5])
isdoublefreethree_position(yx , grid8, stone)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 2. 2. 2. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

True

In [12]:
grid9 = np.zeros((10,10))
grid9[[2, 3, 5, 5],[7, 6, 6, 7]] = 1
grid9

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [16]:
isdoublefreethree_position(np.array([4,5]) , grid9, stone)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

False

In [26]:
grid10 = np.zeros((10,10))
grid10[[2, 3, 3, 2],[2, 3, 5, 6]] = 1
grid10[[1], [7]] = -1
grid10

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [27]:
isdoublefreethree_position(np.array([4,4]) , grid10, stone)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

False

In [580]:
isdoublefreethree_position([4, 4], grid10)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

True

In [59]:
test = np.array([[1, 1, 0, 0, 0]])

kernel = np.array([[1, 1, 1, 1],
                   [1, 1, 1, 1],
                   [0, 1, 1, 1],
                   [0, 0, 1, 1],
                   [0, 0, 0, 1]])

In [60]:
np.dot(test, kernel)

array([[2, 2, 2, 2]])

In [17]:
test

array([[0., 1., 1., 0., 0.]])

In [55]:
res = np.array([[-1, -1, -1, -1, -1]])
for _ in range(10):
    res = np.append(res, np.random.randint(low=0, high=10, size=(1,5)), axis=0)

In [56]:
res

array([[-1, -1, -1, -1, -1],
       [ 6,  0,  0,  6,  3],
       [ 0,  2,  4,  3,  1],
       [ 5,  0,  0,  9,  9],
       [ 9,  9,  1,  1,  6],
       [ 9,  2,  9,  4,  1],
       [ 1,  3,  8,  4,  4],
       [ 5,  6,  6,  8,  0],
       [ 8,  7,  3,  6,  0],
       [ 9,  7,  6,  4,  9],
       [ 8,  6,  4,  1,  2]])

In [62]:
res[1:2, 2:4]

array([[0, 6]])

In [65]:
(res / np.arange(1, 6)).astype('int8')

array([[-1,  0,  0,  0,  0],
       [ 6,  0,  0,  1,  0],
       [ 0,  1,  1,  0,  0],
       [ 5,  0,  0,  2,  1],
       [ 9,  4,  0,  0,  1],
       [ 9,  1,  3,  1,  0],
       [ 1,  1,  2,  1,  0],
       [ 5,  3,  2,  2,  0],
       [ 8,  3,  1,  1,  0],
       [ 9,  3,  2,  1,  1],
       [ 8,  3,  1,  0,  0]], dtype=int8)

In [138]:
res = np.array([[-1, -1]])
for _ in range(10):
    res = np.append(res, np.random.randint(low=0, high=3, size=(1,2)), axis=0)

In [139]:
res = np.delete(res, 0, 0)
res

array([[1, 1],
       [2, 1],
       [1, 2],
       [0, 0],
       [0, 2],
       [1, 1],
       [2, 2],
       [1, 0],
       [1, 1],
       [0, 2]])

In [140]:
np.unique(res, axis=0)

array([[0, 0],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 1],
       [2, 2]])

In [141]:
i = 0
while i < res.shape[0]:
    print(f"res[{i}]: ", res[i])
    mask = (res == res[i]).all(axis=1)
    mask[0] = False
    print("mask:", mask)
    res = np.delete(res, mask, axis=0)
    i += 1

mask: [False False False False False  True False False  True False]
mask: [False  True False False False False False False]
mask: [False False  True False False False False]
mask: [False False False  True False False]
mask: [False False  True False  True]


In [133]:
res

array([[0, 0],
       [1, 0]])

In [83]:
res[1]

array([0, 2])

In [86]:
tmp

array([[ 1,  0],
       [ 1,  1],
       [ 2,  0],
       [ 2,  1],
       [ 3,  0],
       [ 3,  1],
       [ 6,  0],
       [ 6,  1],
       [ 8,  0],
       [ 8,  1],
       [10,  0],
       [10,  1],
       [11,  0],
       [12,  1],
       [15,  0],
       [15,  1],
       [16,  0],
       [16,  1],
       [17,  0],
       [19,  1],
       [20,  0],
       [20,  1]])

In [93]:
tmp[2:,0]

array([ 2,  2,  3,  3,  6,  6,  8,  8, 10, 10, 11, 12, 15, 15, 16, 16, 17,
       19, 20, 20])

## Elaboration du scoreboard

In [4]:
SIZE = 10

kernel = np.array([[1, 0, 1, 0, 1],
                   [0, 1, 1, 1, 0],
                   [1, 1, 1, 1, 1],
                   [0, 1, 1, 1, 0],
                   [1, 0, 1, 0, 1]])

scoreboard = np.zeros((SIZE,SIZE))

grid  = np.zeros((SIZE,SIZE))

nb_white = np.random.randint(35,40)
nb_black = 0
while nb_black <= 0:
	nb_black = nb_white + np.random.randint(-4, 4)

ii = jj = 0
while ii < nb_white:
    x, y = list(np.random.randint(0, SIZE, size = 2))
    grid[y, x] = 1
    ii += 1

while jj < nb_black:
    x, y = list(np.random.randint(0, SIZE, size = 2))
    grid[y, x] = -1
    jj += 1

possible_pos = np.argwhere(grid != 0)
curr_pos = np.array(possible_pos[np.random.randint(0, len(possible_pos) -1, size = 1)][0])

print(f"nb_white = {nb_white} -- nb_black = {nb_black} -- cur_pos = {curr_pos}")
grid

nb_white = 37 -- nb_black = 38 -- cur_pos = [0 5]


array([[ 0.,  1., -1., -1.,  0., -1.,  0.,  0., -1.,  0.],
       [ 0., -1.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0., -1.,  1.,  0., -1., -1.,  0.,  1.],
       [ 0., -1.,  0., -1., -1.,  0.,  0., -1., -1.,  0.],
       [-1.,  0.,  0.,  1.,  0.,  1., -1., -1.,  0.,  1.],
       [-1.,  0.,  0.,  1., -1.,  0., -1.,  0.,  0., -1.],
       [-1.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.],
       [-1.,  1.,  0.,  0., -1.,  1., -1., -1.,  0., -1.],
       [-1.,  0.,  1.,  0., -1.,  0.,  1.,  0.,  0.,  1.],
       [ 0., -1.,  0.,  1.,  0.,  0.,  0.,  0., -1.,  1.]])

In [5]:
extend_grid = np.pad(grid, 4, "constant", constant_values = 0)
extend_scoreboard = np.pad(scoreboard, 2, "constant", constant_values = 0)

In [6]:
curr_pos

array([0, 5])

In [7]:
coordy = curr_pos[0] + 4
coordx = curr_pos[1] + 4
extend_grid[coordy-2:coordy+3, coordx-2:coordx+3]


array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [-1.,  0., -1.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.],
       [-1.,  1.,  0., -1., -1.]])

In [11]:
#extend_scoreboard[curr_pos[0]:curr_pos[0]+5,curr_pos[1]:curr_pos[1]+5] = convolve2d(extend_grid[coordy-2:coordy+3, coordx-2:coordx+3], kernel, "same")
convolve2d(extend_grid[coordy-4:coordy+5, coordx-4:coordx+5], kernel, "valid")

array([[-1., -1., -2., -1., -1.],
       [-4., -1., -2.,  0.,  0.],
       [-3., -3., -3., -2., -2.],
       [-5., -3., -3., -4., -3.],
       [-1., -5., -3., -4., -3.]])

In [94]:
extend_scoreboard

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [-1.,  0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [-1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [-1.,  0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 

In [95]:
scoreboard = extend_scoreboard[2:-2, 2:-2]

In [96]:
scoreboard

array([[-1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])